In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as ta
import seaborn as sns
import matplotlib.dates as mdates


In [ ]:
data = pd.read_csv("C:\WWI2021\Semester 6\Machine Learning Project\LSTM_BTC\data\BTC-USD.csv")
#data

In [ ]:
# Berechnen der Korrelationsmatrix
corr = data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='Blues', mask=mask, cbar_kws={'label': 'Korrelationskoeffizient'},
            linewidths=0.5, linecolor='white')
plt.title('Korrelationsmatrix der Features', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
data_prep = data.drop(['Open', 'High', 'Low','Adj Close', 'Volume'], axis=1)

# Überprüfung des aktualisierten DataFrames
print(data_prep.head())


In [ ]:
#Neue Feature hinzufügen

# ersten 150 Tage nicht nutzbar da der ema150 nicht berechnet werden kann
data_prep['RSI']=ta.rsi(data_prep.Close, length=15)
data_prep['EMA20']=ta.ema(data_prep.Close, length=20)
data_prep['EMA100']=ta.ema(data_prep.Close, length=100)
data_prep['EMA150']=ta.ema(data_prep.Close, length=150)
data_prep

In [ ]:
data_prep['RSI']=ta.rsi(data_prep.Close, length=15)
data_prep['EMA20']=ta.ema(data_prep.Close, length=20)
data_prep['EMA100']=ta.ema(data_prep.Close, length=100)
data_prep['EMA150']=ta.ema(data_prep.Close, length=150)

In [ ]:
import pandas as pd
import ta

# Simple Moving Average (SMA)
data_prep['SMA50'] = ta.trend.SMAIndicator(close=data_prep['Close'], window=50).sma_indicator()

# Momentum Indicator
data_prep['Momentum'] = ta.momentum.ROCIndicator(close=data_prep['Close'], window=10).roc()

# Rate of Change (ROC)
data_prep['ROC'] = ta.momentum.ROCIndicator(close=data_prep['Close'], window=12).roc()

# Bollinger Bands
bollinger = ta.volatility.BollingerBands(close=data_prep['Close'], window=20, window_dev=2)
data_prep['Bollinger_Middle'] = bollinger.bollinger_mavg()
data_prep['Bollinger_Upper'] = bollinger.bollinger_hband()
data_prep['Bollinger_Lower'] = bollinger.bollinger_lband()

# Moving Average Convergence Divergence (MACD)
macd = ta.trend.MACD(close=data_prep['Close'], window_slow=26, window_fast=12, window_sign=9)
data_prep['MACD'] = macd.macd()

# Zielvariablen für die Vorhersage erstellen
data_prep['Percent_Change_Next_Day'] = data_prep['Close'].pct_change(1).shift(-1)
data_prep['Percent_Change_Next_10_Days'] = data_prep['Close'].pct_change(10).shift(-10)

# Optional: Drop NaN values if necessary
data_prep.dropna(inplace=True)
data_prep



In [ ]:
# Berechnen der Korrelationsmatrix
corr = data_prep[['Close', 'RSI', 'EMA20', 'EMA100', 'EMA150', 'SMA50', 'Momentum', 'ROC', 'Bollinger_Middle', 'Bollinger_Upper', 'Bollinger_Lower', 'MACD']].corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='Blues', mask=mask, cbar_kws={'label': 'Korrelationskoeffizient'},
            linewidths=0.5, linecolor='white')
plt.title('Korrelationsmatrix der Features', fontsize=14, fontweight='bold')
plt.show()

In [ ]:
data_prep['TargetNextClose'] = data_prep['Close'].shift(-1)
data_prep

In [ ]:
data_clean = data_prep
data_clean.dropna(inplace=True)
data_clean.reset_index(drop=True, inplace=True)

In [ ]:
data_clean.drop(['Date', 'RSI', 'Momentum', 'ROC'], axis=1, inplace=True)

data_clean.dropna(inplace=True)


In [ ]:
data_clean

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

# Runden der 'Low' und 'High' Werte auf zwei Nachkommastellen
data['Low'] = data['Low'].round(2)
data['High'] = data['High'].round(2)

# Erstellung des Plots
plt.figure(figsize=(14, 7))
plt.plot(data['Date'], data['Close'], label='Close Price', color='royalblue')

# Höchst- und Tiefstpreisen hervorheben
max_price = data['High'].max()
min_price = data['Low'].min()
max_date = data['Date'][data['High'].idxmax()]
min_date = data['Date'][data['Low'].idxmin()]
plt.scatter(max_date, max_price, color='red', label=f'Highest Price: {max_price:.2f} $')
plt.scatter(min_date, min_price, color='green', label=f'Lowest Price: {min_price:.2f} $')

# Beschriftungen und Titel
plt.title('Bitcoin Closing Prices Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=14)
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)
plt.ylabel('Close Price (USD)', fontsize=14)
plt.legend()

ax.grid(True, linestyle='-', linewidth='0.5', color='lightgray')  # Lichtgraues Gitter hinzufügen
ax.set_yticks(range(0, int(data['Close'].max()), 10000))  # Y-Achsen-Ticks in 10.000 $ Schritten

# Anzeigen des Plots
plt.show()